In [20]:
import pandas as pd

pd.set_option('display.max_columns', None)
df = pd.read_csv('data/telecom_churn.csv', sep=',')

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [21]:
df.shape

(7043, 21)

In [22]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [23]:
# Vericar valores nulos
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [24]:
# Verificar as categorias únicas
set(df['StreamingTV'])

{'No', 'No internet service', 'Yes'}

# Pipeline de tratamento dos dados categóricos e limpeza da base

In [25]:
# Map para variaveis binarias
bin_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']

for col in bin_cols:
    df[col] = df[col].map({'Yes': 1, 'No': 0})

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,1,0,1,0,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,1,Electronic check,29.85,29.85,0
1,5575-GNVDE,Male,0,0,0,34,1,No,DSL,Yes,No,Yes,No,No,No,One year,0,Mailed check,56.95,1889.5,0
2,3668-QPYBK,Male,0,0,0,2,1,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,1,Mailed check,53.85,108.15,1
3,7795-CFOCW,Male,0,0,0,45,0,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,Female,0,0,0,2,1,No,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Electronic check,70.70,151.65,1


In [26]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

one_hot_cols = ['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod', 'gender', 
                'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingMovies', 'StreamingTV']
num_cols = ['tenure', 'MonthlyCharges']
drop_cols = ['customerID', 'TotalCharges'] + one_hot_cols + num_cols

# Pipeline de transformação
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('onehot', OneHotEncoder(drop='first'), one_hot_cols)
    ],
    remainder='drop'  # descarta as outras colunas
)

pipe = Pipeline([
    ('preprocessor', preprocessor)
])

# Aplicar transformação
transformed_array = pipe.fit_transform(df)

# Nome das colunas
num_scaled_cols = num_cols
one_hot_encoded_cols = pipe.named_steps['preprocessor'] \
                           .named_transformers_['onehot'] \
                           .get_feature_names_out(one_hot_cols)
all_cols = list(num_scaled_cols) + list(one_hot_encoded_cols)

# Transformar em DataFrame
transformed_df = pd.DataFrame(transformed_array, columns=all_cols, index=df.index)

# Concatenar com as colunas que não queremos transformar
df_final = pd.concat([df.drop(columns=drop_cols), transformed_df], axis=1)

df_final.head()


,SeniorCitizen,Partner,Dependents,PhoneService,PaperlessBilling,Churn,tenure,MonthlyCharges,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,gender_Male,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No internet service,TechSupport_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,StreamingTV_No internet service,StreamingTV_Yes
0,0,1,0,0,1,0,-1.277445,-1.160323,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,1,0,0,0.066327,-0.259629,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,1,1,1,-1.236724,-0.362660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0.514251,-0.746535,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0,0,0,1,1,1,-1.236724,0.197365,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
df_final.shape

(7043, 30)

In [28]:
# Verificar valores nulos
df_final.isna().any()

SeniorCitizen                            False
Partner                                  False
Dependents                               False
PhoneService                             False
PaperlessBilling                         False
Churn                                    False
tenure                                   False
MonthlyCharges                           False
MultipleLines_No phone service           False
MultipleLines_Yes                        False
InternetService_Fiber optic              False
InternetService_No                       False
Contract_One year                        False
Contract_Two year                        False
PaymentMethod_Credit card (automatic)    False
PaymentMethod_Electronic check           False
PaymentMethod_Mailed check               False
gender_Male                              False
OnlineSecurity_No internet service       False
OnlineSecurity_Yes                       False
OnlineBackup_No internet service         False
OnlineBackup_

# Separando os dados em treino e teste

In [29]:
from sklearn.model_selection import train_test_split

X = df_final.drop(columns='Churn')
y = df_final['Churn']

# Equilibrando a target

In [30]:
df_final['Churn'].value_counts(normalize=True) * 100

Churn
0    73.463013
1    26.536987
Name: proportion, dtype: float64

In [31]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [32]:
y_resampled.value_counts(normalize=True) * 100

Churn
0    50.0
1    50.0
Name: proportion, dtype: float64

In [33]:
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Criando e treinando o modelo de Regressão Logística
modelo = LogisticRegression()

scores = cross_val_score(
    modelo,
    x_train, 
    y_train, 
    cv=10, # Quantidade da validacooes cruzadas K FOLDERS
    scoring='accuracy'
    )

print(scores)
print("Acurácia média com validação cruzada:", scores.mean())

modelo.fit(x_train, y_train) # .fit -> treinamento de modelo

[0.78502415 0.77536232 0.75603865 0.80434783 0.7294686  0.76811594
 0.75120773 0.76570048 0.74879227 0.75120773 0.75845411 0.80193237
 0.79227053 0.76328502 0.76570048 0.7826087  0.75120773 0.76570048
 0.79661017 0.79903148]
Acurácia média com validação cruzada: 0.7706033383631026


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [35]:
y_pred = modelo.predict(x_test)
y_pred

array([1, 0, 0, ..., 1, 1, 0], shape=(2070,))

In [36]:
y_proba = modelo.predict_proba(x_test)[:, 1]  # probabilidade da classe 1 (Churn)
y_proba

array([0.65000576, 0.3202382 , 0.43847323, ..., 0.84352541, 0.65099101,
       0.43327709], shape=(2070,))

In [37]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy:.2f}")

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


Acurácia: 0.79
[[767 254]
 [188 861]]
              precision    recall  f1-score   support

           0       0.80      0.75      0.78      1021
           1       0.77      0.82      0.80      1049

    accuracy                           0.79      2070
   macro avg       0.79      0.79      0.79      2070
weighted avg       0.79      0.79      0.79      2070



## TESTE